In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [43]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [44]:
df.head()

,Title,PositiveReview,NegativeReview,Score,GuestName,GuestCountry,RoomType,NumberOfNights,VisitDate,GroupType,PropertyResponse
0,Wonderful place to stay.,"New, comfortable apartments, close to the airp...",Nothing at all.,10.0,Olga,Norway,Budget Twin Room,1 night,June 2022,Solo traveler,NaN
1,It was superb,We had a really pleasant stay! The staff was v...,NaN,10.0,Iwona,Poland,Double Room,3 nights,December 2022,Family,NaN
2,Very Good,the location is great and near the airport. bu...,NaN,8.0,Ruijia,Sweden,Double Room,1 night,December 2022,Solo traveler,NaN
3,Wonderful,Great stuff\nGreat Quality/price\nClean,NaN,9.0,Theprincem,United Kingdom,Double Room with Balcony,2 nights,September 2022,Solo traveler,NaN
4,"Fantastic value for a new, modern and spotless...","Clean and modern with very comfortable beds, i...",NaN,10.0,M,Switzerland,Family Suite with Balcony,1 night,October 2022,Family,NaN


In [85]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [90]:
llm = ChatOpenAI(temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
"Can you prepare a apology \
message about NegativeReview \
addressing the user by name? Follow this file:"
"\n\n{Otels_Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="ApologyMessage"
                    )


In [91]:
second_prompt = ChatPromptTemplate.from_template(
    "Translate the following apology message to Turkish"
    "\n\n{ApologyMessage}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="Turkish"
                    )

In [92]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["Otels_Review"],
    output_variables=["ApologyMessage", "Turkish"],
    verbose=True
)

In [95]:

Otels_Review  = df.iloc[62]
overall_chain(Otels_Review)




> Entering new SequentialChain chain...

> Finished chain.


{'Otels_Review': Title               Was by far too short.I’ll be back in February....
 PositiveReview      It’s been renovated to a exceptionally high st...
 NegativeReview      I cannot find a single negative point…which is...
 Score                                                            10.0
 GuestName                                                      Philip
 GuestCountry                                     United Arab Emirates
 RoomType                                             Budget Twin Room
 NumberOfNights                                               2 nights
 VisitDate                                                October 2022
 GroupType                                               Solo traveler
 PropertyResponse                                                  NaN
 Name: 62, dtype: object,
 'ApologyMessage': 'Dear Philip,\n\nI hope this message finds you well. I wanted to reach out to you personally in regards to your recent stay at our property and your review on

In [96]:
Otels_Review  = df.iloc[62]
print(Otels_Review)

Title               Was by far too short.I’ll be back in February....
PositiveReview      It’s been renovated to a exceptionally high st...
NegativeReview      I cannot find a single negative point…which is...
Score                                                            10.0
GuestName                                                      Philip
GuestCountry                                     United Arab Emirates
RoomType                                             Budget Twin Room
NumberOfNights                                               2 nights
VisitDate                                                October 2022
GroupType                                               Solo traveler
PropertyResponse                                                  NaN
Name: 62, dtype: object


In [97]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [98]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [99]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [100]:
llm = ChatOpenAI(temperature=0)

In [101]:

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [102]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [103]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [104]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [105]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [106]:
chain.run("What is resistor?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is a resistor?'}
> Finished chain.


'A resistor is an electrical component that is used to resist the flow of electric current in a circuit. It is typically made of a material that has high resistance to the flow of electricity, such as carbon or metal. Resistors are used in a variety of electronic devices to control the amount of current flowing through a circuit and to protect other components from damage due to excessive current. They are also used to create voltage drops and to divide voltage in a circuit.'

In [107]:
chain.run("what is 2123*123123")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2123*123123'}
> Finished chain.


'The answer to 2123*123123 is 261,383,529.'

In [108]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for the synthesis of proteins, which are essential for the structure and function of cells. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next. Therefore, every cell in our body needs DNA to carry out its specific functions and to maintain the integrity of the organism as a whole.'

In [109]:
chain.run("What is the Maven?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is the Maven?'}
> Finished chain.


"Maven is a build automation tool used primarily for Java projects. It helps in managing the project's build, dependencies, and documentation. Maven uses a project object model (POM) file to define the project structure, dependencies, and build process. It also provides a standard directory layout for the project, making it easier for developers to understand and navigate the project structure. Maven can be used from the command line or integrated into an IDE. It is widely used in the Java community and is known for its ease of use and flexibility."

In [110]:
chain.run("What is the Maven in Java Proggraming Lanugage?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the Maven in Java Programming Language?'}
> Finished chain.


'Maven is a build automation tool used primarily for Java projects. It helps manage project dependencies, build processes, and project documentation. Maven uses a Project Object Model (POM) file to define the project structure, dependencies, and build process. It also provides a standard directory layout for projects, making it easier for developers to understand and navigate the project structure. Maven is widely used in the Java community and is supported by many popular IDEs such as Eclipse and IntelliJ IDEA.'